In [57]:
import numpy as np
import pandas as pd

In [58]:
df = pd.read_csv("post_feature_selection.csv")

In [59]:
df.head()

property_type  sector  bedroom  bathroom  balcony  agePossession  \
0            0.0    79.0      2.0       2.0      1.0            3.0   
1            0.0    34.0      2.0       2.0      1.0            2.0   
2            0.0   111.0      2.0       2.0      3.0            1.0   
3            0.0    69.0      2.0       2.0      2.0            4.0   
4            0.0   105.0      2.0       2.0      3.0            4.0   

   built_up_area  servant room  store room  furnishing_type  luxury_class  \
0         1000.0           0.0         0.0              0.0           1.0   
1          722.0           0.0         0.0              1.0           1.0   
2          661.0           0.0         0.0              0.0           1.0   
3         1333.0           0.0         0.0              0.0           2.0   
4         1217.0           0.0         0.0              0.0           1.0   

   floor_class  price  
0          2.0   0.45  
1          1.0   0.50  
2          0.0   0.40  
3          1.0   1.47  
4          2.0   0.70

In [60]:
df.shape

(3577, 13)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3577 entries, 0 to 3576
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   property_type    3577 non-null   float64
 1   sector           3577 non-null   float64
 2   bedroom          3577 non-null   float64
 3   bathroom         3577 non-null   float64
 4   balcony          3577 non-null   float64
 5   agePossession    3577 non-null   float64
 6   built_up_area    3577 non-null   float64
 7   servant room     3577 non-null   float64
 8   store room       3577 non-null   float64
 9   furnishing_type  3577 non-null   float64
 10  luxury_class     3577 non-null   float64
 11  floor_class      3577 non-null   float64
 12  price            3577 non-null   float64
dtypes: float64(13)
memory usage: 363.4 KB


In [62]:
### dividing into x and y

x = df.drop(columns = ["price"])
y = df["price"]

In [63]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [64]:
col = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_class', 'floor_class']

In [65]:
y_trans = np.log1p(y)

In [66]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedroom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first'), col)
    ], 
    remainder='passthrough'
)

In [67]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [68]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, x, y_trans, cv=kfold, scoring='r2')

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1006: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 345, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\USER\anaconda3\Lib\site-packages\sklearn\utils\_response.py", line 238, in _get_response_values
    y_pred, pos_label = prediction_method(X), None
                        ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\USER\anaconda3\Lib\site-packages\sklearn\pi

In [69]:
scores

array([0.89841157, 0.90447776, 0.87555346, 0.90843028,        nan,
       0.87647433,        nan, 0.86989439, 0.87692071, 0.85816844])

In [25]:
scores.mean()

nan

In [26]:
scores.std()

nan

In [29]:
#train-test-split

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y_trans,test_size=0.2,random_state=42)

In [31]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedroom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_class',
                                                   'floor_class'])])),
                ('regressor', SVR())])

In [33]:
y_pred = pipeline.predict(x_test)

In [35]:
y_pred = pipeline.predict(x_test)

In [36]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

1.5084797467203646